In [ ]:
!pip install catboost
!pip install kaggle

import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 kaggle.json

!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip -o udea*.zip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 2.21GB/s]
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


Se instalan las librerías necesarias:

catboost: algoritmo principal del modelo.

kaggle: para descargar los datos del concurso.

Se configura el entorno para que Kaggle pueda leer el archivo kaggle.json con las credenciales.

Se cambian los permisos del archivo por seguridad.

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

train.shape, test.shape


((692500, 21), (296786, 20))

Se descarga el dataset oficial del concurso.

*-o* permite sobreescribir archivos existentes al descomprimir.

In [ ]:
TARGET = "RENDIMIENTO_GLOBAL"
ID = "ID"

X = train.drop(columns=[TARGET, ID])
y = train[TARGET]

X_test_final = test.drop(columns=[ID])

# Convertir categóricas a string
cat_cols = X.select_dtypes(include="object").columns.tolist()
for c in cat_cols:
    X[c] = X[c].astype(str)
    X_test_final[c] = X_test_final[c].astype(str)

# SAMPLE estratificado (200k)
from sklearn.model_selection import train_test_split

X_sample, _, y_sample, _ = train_test_split(
    X, y, train_size=200_000, stratify=y, random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_sample, y_sample,
    test_size=0.20, stratify=y_sample, random_state=42
)

X_train.shape, X_valid.shape


((160000, 19), (40000, 19))

Se cargan los archivos en DataFrame.

Se imprime el tamaño para confirmar que los datos cargaron correctamente.

Se define:

TARGET: la variable a predecir (clasificación multiclase).

ID: identificador único que no debe usarse como predictor.

Se separan:

X: características predictoras.

y: etiquetas reales.

X_test_final: características del dataset de prueba.

CatBoost acepta variables categóricas en formato string.
Esto garantiza consistencia y evita errores en el modelo.

Se reduce el dataset a una muestra de 200,000 registros, preservando las proporciones del target (stratify=y).

Esto se hace para acelerar el entrenamiento sin perder representatividad.

Se usa un split 80% entrenamiento / 20% validación.

Se mantiene estratificación para evitar sesgos de distribución.

In [ ]:
from catboost import Pool

train_pool = Pool(X_train, y_train, cat_features=cat_cols)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_cols)
test_pool  = Pool(X_test_final,    cat_features=cat_cols)


CatBoost utiliza su estructura propia (Pool) que permite:

Manejo eficiente de categóricas

Optimizaciones internas

Entrenamiento más rápido y robusto

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="Accuracy",
    random_seed=42,
    iterations=100,        # MUY rápido
    learning_rate=0.25,    # Aumentado para compensar pocas iteraciones
    depth=6,
    subsample=0.8,
    rsm=0.8,
    bootstrap_type="Bernoulli",
    od_type="Iter",
    od_wait=20,
    verbose=10
)

model.fit(train_pool, eval_set=valid_pool, use_best_model=True)


0:	learn: 0.3684438	test: 0.3703250	best: 0.3703250 (0)	total: 668ms	remaining: 1m 6s
10:	learn: 0.4159375	test: 0.4183500	best: 0.4183500 (10)	total: 6.7s	remaining: 54.2s
20:	learn: 0.4239000	test: 0.4219500	best: 0.4225250 (18)	total: 12.1s	remaining: 45.6s
30:	learn: 0.4280813	test: 0.4277250	best: 0.4277250 (30)	total: 17.8s	remaining: 39.5s
40:	learn: 0.4300062	test: 0.4287250	best: 0.4291250 (36)	total: 23.3s	remaining: 33.5s
50:	learn: 0.4328562	test: 0.4295750	best: 0.4296500 (45)	total: 28.7s	remaining: 27.6s
60:	learn: 0.4370688	test: 0.4310500	best: 0.4312000 (58)	total: 34.6s	remaining: 22.1s
70:	learn: 0.4389375	test: 0.4306000	best: 0.4313500 (61)	total: 40s	remaining: 16.3s
80:	learn: 0.4402937	test: 0.4304500	best: 0.4313500 (61)	total: 46s	remaining: 10.8s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.43135
bestIteration = 61

Shrink model to first 62 iterations.


Parámetro	Función
* loss_function="MultiClass"	Indica que es un problema de clasificación multiclase.
* eval_metric="Accuracy"	Métrica para evaluar el desempeño.
* iterations=100	Entrenamiento muy rápido (pocas iteraciones).
* learning_rate=0.25	Se aumenta para compensar pocas iteraciones.
* depth=6	Profundidad de los árboles (controla complejidad).
* subsample=0.8	Hace el entrenamiento más robusto al ruido.
* rsm=0.8	Random Subspace Method, reduce overfitting.
* bootstrap_type="Bernoulli"	Muestreo estocástico eficiente.
* od_wait=20	Early stopping basado en validación.
* verbose=10	Imprime progreso cada 10 iteraciones.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

pred_valid = model.predict(X_valid).flatten()
acc = accuracy_score(y_valid, pred_valid)

print("Accuracy VALID:", acc)
print(classification_report(y_valid, pred_valid))


Accuracy VALID: 0.43135
              precision    recall  f1-score   support

        alto       0.54      0.64      0.58     10144
        bajo       0.46      0.55      0.50      9992
  medio-alto       0.32      0.26      0.29      9913
  medio-bajo       0.34      0.27      0.30      9951

    accuracy                           0.43     40000
   macro avg       0.41      0.43      0.42     40000
weighted avg       0.42      0.43      0.42     40000



Se miden las predicciones en validación.

classification_report entrega métricas detalladas:

* precision

* recall

* F1-score

* soporte por clase

In [ ]:
test_pred = model.predict(X_test_final).flatten()

submission = pd.DataFrame({
    "ID": test[ID],
    "RENDIMIENTO_GLOBAL": test_pred
})

submission.to_csv("my_submission.csv", index=False)
submission.head()


,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


Se generan predicciones sobre el set de test.

Se construye el archivo final con el formato requerido.

Se exporta como my_submission.csv, listo para subir a Kaggle.

In [ ]:
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f my_submission.csv -m "versión final catboost"

100% 4.04M/4.04M [00:00<00:00, 9.46MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia